In [2]:
%pip install matplotlib ipywidgets numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 77.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 76.3 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 108.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 75.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 120.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
#par Brian BERNARD, pour toutes questions : brian.bernard@ens-paris-saclay.fr
%matplotlib inline
from ipywidgets import interactive
import matplotlib.pyplot as plt
import numpy as np
from numpy.polynomial import polynomial as P

verbose = False
#On considère (arbitrairement) la transformation à P fixée, mais de fait le volume de l'est pas. Un exercice peut être de modifier ce script (et donc les expressions physiques associées) pour prendre en entrée le volume et observer la pression varier entre l'état initial et final.

def get_sub(x):
    normal = "ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789+-=()"
    sub_s = "ₐ₈CDₑբGₕᵢⱼₖₗₘₙₒₚQᵣₛₜᵤᵥwₓᵧZₐ♭꜀ᑯₑբ₉ₕᵢⱼₖₗₘₙₒₚ૧ᵣₛₜᵤᵥwₓᵧ₂₀₁₂₃₄₅₆₇₈₉₊₋₌₍₎"
    res = x.maketrans(''.join(normal), ''.join(sub_s))
    return x.translate(res)


"""
notes préliminaires : pour cette étude de N2+3H2 = 2NH3, tous gazeux :
Delta_r_G = R*T ln(Q/K)
Delta_r_G = Delta_r_H - T*Delta_r_S
ΔrH°298 = – 92,2 kJ/mole
ΔrS°298 = – 0.189 kJ/mole/K
"""

def avancement(K,nH,nN,nNH,nHe,Ptot):
    """ K° = (nNH+2ksi)² ntot²/(Ptot² *(nN-ksi)*(nH-3ksi)**3) , Ptot en bar
    On détermine alors les racines de ce polynôme constituant un avancement acceptable physiquement"""
    ntot = nH+nN+nNH+nHe
    A = K*(Ptot**2)#intermédiaire de calcul
    B = ntot #intermédiaire de calcul, pas au carré car mis au carré dans polypow

    coeff = P.polyadd(P.polymul([A,-A*nN],P.polypow([-3,nH],3)),P.polymul(P.polypow([B*2,B*nNH],2),P.polypow([-2,ntot],2)))
    racinescmplx = np.roots(coeff).tolist()
    racinescplx = [complex(round(k.real,8),round(k.imag,8)) for k in racinescmplx]
    racines = [np.real(n) for n in racinescplx if np.imag(n) == 0]
    if verbose :
      print(racinescplx)  #afficher toutes les racines trouvées, en cas de problème avec la solution affichée

    racinepos = [k for k in racines if (k>-nNH/2  and k< min(nH/3,nN))] #pas de compensation d'erreurs flottantes, à modifier
    if racinepos == [] :
        print("Erreur : l'équation polynomiale d'équilibre n'a pas de racines réelles acceptables")
        print(racinepos)
        return(0)
    elif len(racinepos)>1:
        print("Erreur : l'équation polynomiale d'équilibre a pluss d'une racine acceptable")
        print(racinepos)
        return(racinepos[0])
    else :
        return(racinepos[0])


def f(T,Ptot,nH2,nN2,nNH3,nHe):


    Delta_r_G = -92.2+T*0.189 #dans l'approximation d'Ellingham
    print("ΔrG° (T)= ",round(-92.2+T*0.189,2),"kJ/mol")
    K = np.exp(-Delta_r_G*1000/(8.314*T)) #on calcule K qui ne dépend que de T
    print("K° (T)= ",round(K,4))

    ksi = avancement(K,nH2,nN2,nNH3,nHe,Ptot) #par recherche des racines du polynôme fourni.
    print("avancement :", ksi)
    volumei = (nHe+nNH3+nN2+nH2)*8.317*T/(Ptot*(10**(5-3))) #volume, en L
    volumef = (nHe+nNH3+nN2+nH2-2*ksi)*8.317*T/(Ptot*(10**(5-3)))

    print( "état initial ; état d'équilibre : ")
    print('nH2 : ',nH2 ,'\t',round(nH2-3*ksi,2) , ' mol')
    print('nN2 : ',nN2 ,'\t',round(nN2-ksi,2) , ' mol')
    print('nNH3 : ',nNH3 ,'\t',round(nNH3+2*ksi,2) , ' mol')
    print('V : ',round(volumei,0),'\t',round(volumef,0), ' L')


#actualisation des paramètres interactifs
interactive_plot = interactive(f, T=(150, 1000,50),Ptot=(1, 190,9),nH2=(0, 10,1),nN2=(0, 10,1),nNH3=(0, 10,1),nHe=(0, 10,1))
output = interactive_plot.children[-1]
output.layout.height = '350px'
interactive_plot


interactive(children=(IntSlider(value=550, description='T', max=1000, min=150, step=50), IntSlider(value=91, d…